# Write Component from YAML netlist


Note that you define the connections as `instance_source.port -> instance_destination.port` so the order is important and therefore you can only change the position of the `instance_destination`

In [ ]:
import pp

netlist = """
instances:
    CP1:
      component: mmi1x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi1x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
    arm_bot:
        component: mzi_arm

placements:
    arm_bot:
        mirror: [0,0,0,10]
ports:
    W0: CP1,W0
    E0: CP2,W0

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E0: arm_top,E0
"""

c = pp.component_from_yaml(netlist)
pp.show(c)
pp.plotgds(c)

## Adjust component settings

We can reduce the length of each of the arms

In [ ]:
import pp

netlist = """
instances:
    CP1:
      component: mmi1x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi1x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
    arm_bot:
        component: mzi_arm
        settings:
            L0: 0
            DL: 10

placements:
    arm_bot:
        mirror: [0,0,0,10]
ports:
    W0: CP1,W0
    E0: CP2,W0

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E0: arm_top,E0
"""

c = pp.component_from_yaml(netlist)
pp.show(c)
pp.plotgds(c)

## Swap components 

We can also use 2x2 couplers instead of 1x2 MMIs

In [ ]:
import pp

netlist = """
instances:
    CP1:
      component: mmi2x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi2x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
    arm_bot:
        component: mzi_arm
        settings:
            L0: 0
            DL: 10

placements:
    arm_bot:
        mirror: [0,0,0,10]
ports:
    W0: CP1,W0
    E0: CP2,W0
    W1: CP1,W1
    E1: CP2,W1

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E0: arm_top,E0
"""

c = pp.component_from_yaml(netlist)
pp.show(c)
pp.plotgds(c)

## Exposing more ports

We can also expose more ports, such as the electrical ports, so we can route electrical signals to the circuits.

In [ ]:
import pp

netlist = """
instances:
    CP1:
      component: mmi2x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi2x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
    arm_bot:
        component: mzi_arm
        settings:
            L0: 0
            DL: 10

placements:
    arm_bot:
        mirror: [0,0,0,10]
ports:
    W0: CP1,W0
    E0: CP2,W0
    W1: CP1,W1
    E1: CP2,W1
    E_TOP_0: arm_top,E_0
    E_TOP_1: arm_top,E_1
    E_TOP_2: arm_top,E_2
    E_TOP_3: arm_top,E_3
    E_BOT_0: arm_bot,E_0
    E_BOT_1: arm_bot,E_1
    E_BOT_2: arm_bot,E_2
    E_BOT_3: arm_bot,E_3

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E0: arm_top,E0
"""

c = pp.component_from_yaml(netlist)
pp.show(c)
pp.plotgds(c)

In [ ]:
c.ports

## Custom factories

You can leverage netlist defined components to define more complex circuits

In [ ]:
import pp

@pp.autoname
def mzi_custom(delta_length=0):
    netlist = f"""
instances:
    CP1:
      component: mmi2x2
      settings:
          width_mmi: 4.5
          length_mmi: 10
    CP2:
        component: mmi2x2
        settings:
            width_mmi: 4.5
            length_mmi: 5
    arm_top:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
            with_elec_connections: False
    arm_bot:
        component: mzi_arm
        settings:
            L0: 0
            DL: {delta_length/2}
            with_elec_connections: False

placements:
    arm_bot:
        mirror: [0,0,0,10]
ports:
    W0: CP1,W0
    E0: CP2,W0
    W1: CP1,W1
    E1: CP2,W1

connections:
    arm_bot,W0: CP1,E0 
    arm_top,W0: CP1,E1
    CP2,E0: arm_bot,E0 
    CP2,E0: arm_top,E0
"""
    return pp.component_from_yaml(netlist)


c = mzi_custom(delta_length=10, cache=False)
pp.show(c)
pp.plotgds(c)

In [ ]:
c.ports

In [ ]:
import pp

@pp.autoname
def mzi_custom(delta_length):
    return pp.c.mzi(DL=delta_length/2, coupler_factory=pp.c.mmi2x2)

pp.c.component_type2factory.update(dict(mzi_custom=mzi_custom))
c = pp.c.component_type2factory['mzi_custom'](delta_length=0, cache=False)
pp.plotgds(c)
pp.show(c)
print(c.ports.keys())

In [ ]:
import pp

@pp.autoname
def mzi_filter(delta_lengths=(20, 40, 60), component_type2factory=pp.c.component_type2factory):
    sample = f"""
instances:
    mzi1:
      component: mzi_custom
      settings:
          delta_length: {delta_lengths[0]}

    arm_top1:
        component: mzi_arm
        settings:
            L0: 0
            DL: 0
            with_elec_connections: False
    arm_bot1:
        component: mzi_arm
        settings:
            L0: 0
            DL: {delta_lengths[1]/2}
            with_elec_connections: False

    mzi3:
        component: mzi_custom
        settings:
            delta_length: {delta_lengths[2]}

placements:
    arm_bot1:
        mirror: [0,0,0,10]

ports:
    W0: mzi1,W0
    E0: mzi3,E0
    W1: mzi1,W1
    E1: mzi3,E1

connections:
    arm_bot1,W0: mzi1,E0
    arm_top1,W0: mzi1,E1
    mzi3,W0: arm_bot1,E0
    mzi3,W1: arm_top1,E0
    
"""

    c = pp.component_from_yaml(sample, component_type2factory=component_type2factory)
    return c

c = mzi_filter(cache=False)
pp.show(c)
pp.plotgds(c)